In [ ]:
import rasterio
import numpy as np
from rasterstats import zonal_stats
import geopandas as gpd
import json
import pandas as pd
import openpyxl
import regex


geotiff_path = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\images\\All_Sensor_Courses\\Course_01A_Outline\\2698245e-873a-4aeb-9d76-97353a81172b\PSScene\\20240604_162151_41_24d0_3B_AnalyticMS_SR_clip.tif'

metadata_json_path = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\images\\All_Sensor_Courses\\Course_01A_Outline\\2698245e-873a-4aeb-9d76-97353a81172b\PSScene\\20240604_162151_41_24d0_metadata.json'

ndvi_path = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\images\\All_Sensor_Courses\\Course_01A_Outline\\2698245e-873a-4aeb-9d76-97353a81172b\PSScene\\20240604_162151_41_24d0_3B_AnalyticMS_SR_clip_ndvi.tif'

greens_path = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\greens-geojson\\Course_01A_Greens.geojson"

greens_projected_path = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\greens-geojson\\Course_01A_Greens_Projected.geojson"

output_csv_path = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\output-csv\\Course_01A.csv"

output_xlsx_path = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\output-csv\\Course_01A.xlsx"


In [23]:

# 1) Read the target CRS from the GeoTIFF
with rasterio.open(geotiff_path) as src:
    tif_crs = src.crs

if tif_crs is None:
    raise ValueError("The GeoTIFF does not have a defined CRS. Please set it before proceeding.")

print("Target CRS from TIFF:", tif_crs)

# 2) Load the GeoJSON
gdf = gpd.read_file(greens_path)


# 4) Reproject to the TIFF's CRS
gdf_matched = gdf.to_crs(tif_crs)

# 5) Save as GeoJSON
gdf_matched.to_file(greens_projected_path, driver="GeoJSON")
print(f"Saved reprojected GeoJSON: {greens_projected_path}")


Target CRS from TIFF: EPSG:32615
Saved reprojected GeoJSON: C:\Users\msmurphy\Documents\WinterTurf Planet\Sensor Courses\greens-geojson\Course_01A_Greens_Projected.geojson


In [24]:

with rasterio.open(geotiff_path) as src:
    # Read the Red (Band 3) and NIR (Band 4) bands
    # Rasterio uses 1-based indexing for bands
    red = src.read(3).astype(float)
    nir = src.read(4).astype(float)
    
    # Get the metadata from the source file to use for the output file
    meta = src.meta

# Calculate NDVI
# Use numpy.seterr to ignore division by zero warnings, which are handled by np.where
np.seterr(divide='ignore', invalid='ignore')
ndvi = np.where(
    (nir + red) == 0., # Condition to check for zero division
    0,                # Value to use if condition is true (handles no-data areas)
    (nir - red) / (nir + red) # The NDVI formula
)

# Update the metadata for the new NDVI file
meta.update(
    dtype=rasterio.float32, # NDVI values are floats between -1 and 1
    count=1,                # The output will have only one band
    compress='lzw'          # Optional: adds compression
)

# Write the NDVI array to a new GeoTIFF file
with rasterio.open(ndvi_path, 'w', **meta) as dst:
    dst.write(ndvi.astype(rasterio.float32), 1)

print(f"NDVI calculated and saved to {ndvi_path}")

NDVI calculated and saved to C:\Users\msmurphy\Documents\WinterTurf Planet\Sensor Courses\images\All_Sensor_Courses\Course_01A_Outline\2698245e-873a-4aeb-9d76-97353a81172b\PSScene\20240604_162151_41_24d0_3B_AnalyticMS_SR_clip_ndvi.tif


In [ ]:
#some helper functions for filtering the green numbers and flattening the dictionary

def flatten_dict(d, parent_key="", sep="."):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def parse_green_no(val):
    if val is None:
        return None
    if isinstance(val, (int, float)):
        n = int(val)
        return n if 1 <= n <= 18 else None
    return None



In [32]:

# Load & flatten metadata JSON

with open(metadata_json_path, "r") as f:
    metadata = json.load(f)

if isinstance(metadata, list):
    # If the JSON is a list, take the first item or merge as needed
    # Here we take the first item for a single-row summary
    if not metadata:
        raise ValueError("metadata.json is an empty list.")
    metadata = metadata[0]

if not isinstance(metadata, dict):
    raise ValueError("metadata.json must be an object (or a list of objects).")

meta_flat = flatten_dict(metadata)
meta_df = pd.DataFrame([meta_flat])  # single-row DataFrame


In [36]:
# --- Read raster metadata ---
with rasterio.open(ndvi_path) as src:
    raster_nodata = src.nodata
# print("Raster NoData:", raster_nodata)

# --- Load projected GeoJSON ---
gdf = gpd.read_file(greens_projected_path)

# Ensure the 'green_no' field exists
if "green_no" not in gdf.columns:
    raise KeyError("Field 'green_no' not found in GeoJSON properties.")

# --- Prepare GeoJSON-like shapes with properties ---
shapes = json.loads(gdf.to_json())

# --- Zonal stats: get pixel SUM and COUNT per feature ---
stats = zonal_stats(
    shapes,                # FeatureCollection with geometry + properties
    ndvi_path,
    stats=["sum", "count"],  # sum of pixel values, and number of valid pixels
    nodata=raster_nodata,    # exclude nodata pixels
    geojson_out=True,
    all_touched=False        # set True for more inclusive boundary behavior if desired
)

# --- Build a DataFrame of per-feature results ---
rows = []

for feat in stats:
    props = feat.get("properties", {}) or {}
    gn = parse_green_no(props.get("green_no"))

    sum_val = props.get("sum")
    count_val = props.get("count")

    # Skip features with no valid pixels
    if gn is None or sum_val is None or count_val in (None, 0):
        continue

    rows.append({"green_no": gn, "sum": float(sum_val), "count": int(count_val)})

df = pd.DataFrame(rows)

if df.empty:
    raise ValueError("No valid NDVI pixels found for any feature with a valid green_no in [1..18].")


grouped = df.groupby("green_no", as_index=False).agg(
    total_sum=("sum", "sum"),
    total_count=("count", "sum"),
)
grouped["mean_ndvi"] = grouped["total_sum"] / grouped["total_count"]

# -------------------
# Build columns green_1..green_18 and overall_ndvi
# -------------------
green_cols = {}
for n in range(1, 19):
    val = grouped.loc[grouped["green_no"] == n, "mean_ndvi"]
    green_cols[f"green_{n}"] = (val.iloc[0] if len(val) == 1 else pd.NA)

# Overall pixel-weighted mean across all pixels in any feature
overall_sum = grouped["total_sum"].sum()
overall_count = grouped["total_count"].sum()
overall_ndvi = overall_sum / overall_count

# -------------------
# Combine metadata + NDVI summary into a single-row DataFrame
# -------------------
out_row = {**meta_flat, **green_cols, "overall_ndvi": overall_ndvi}
out_df = pd.DataFrame([out_row])

# Optional: reorder columns (metadata first, then green_1..green_18, then overall_ndvi)
meta_cols = list(meta_flat.keys())
ndvi_cols = [f"green_{n}" for n in range(1, 19)] + ["overall_ndvi"]
ordered_cols = meta_cols + ndvi_cols
out_df = out_df.reindex(columns=ordered_cols)

#configure columns to remove properties header and all geometry info
cols_to_drop = [c for c in out_df.columns if c.startswith("geometry.")]
out_df = out_df.drop(columns=cols_to_drop)

out_df.columns = out_df.columns.str.replace(r'^properties\.', '', regex=True)

#put the date time into local
# 1) Parse to timezone-aware UTC datetime
out_df["acquired_dt"] = pd.to_datetime(out_df["acquired"], utc=True, errors="coerce")

# 2) Convert to local US Central time (handles CST/CDT automatically)
out_df["acquired_dt_local"] = out_df["acquired_dt"].dt.tz_convert("America/Chicago")

# 3) Create date and time columns (local time)
out_df["acquired_date"] = out_df["acquired_dt_local"].dt.date
out_df["acquired_time"] = out_df["acquired_dt_local"].dt.strftime("%H:%M:%S")

# 4)  Drop helper columns
out_df = out_df.drop(columns=["acquired_dt", "acquired_dt_local"])


# Identify green and special columns
green_cols_all = [f"green_{n}" for n in range(1, 19)]
special_cols = set(green_cols_all + ["overall_ndvi", "acquired_date", "acquired_time"])

# Metadata columns = everything else that's not special
meta_cols = [c for c in out_df.columns if c not in special_cols and not c.startswith("green_")]

# Keep only green columns that actually exist (in case some greens are missing)
green_cols_existing = [c for c in green_cols_all if c in out_df.columns]

# Build the desired order:
ordered_cols = meta_cols + ["acquired_date", "acquired_time"] + green_cols_existing + ["overall_ndvi"]

# Finally, reindex to that order (only keep columns that exist)
ordered_cols = [c for c in ordered_cols if c in out_df.columns]
out_df = out_df.reindex(columns=ordered_cols)


# Save outputs
out_df.to_csv(output_csv_path, index=False)
try:
    out_df.to_excel(output_xlsx_path, index=False, engine="openpyxl")
except Exception as e:
    print("Excel export failed:", e)

print("\n=== NDVI summary row ===")
print(out_df.to_string(index=False))
print(f"\nSaved CSV: {output_csv_path}")
print(f"Saved Excel: {output_xlsx_path}")




=== NDVI summary row ===
                     id    type                    acquired  anomalous_pixels  clear_confidence_percent  clear_percent  cloud_cover  cloud_percent  ground_control  gsd  heavy_haze_percent instrument item_type  light_haze_percent  pixel_resolution    provider            published publishing_stage quality_category  satellite_azimuth satellite_id  shadow_percent  snow_ice_percent strip_id  sun_azimuth  sun_elevation              updated  view_angle  visible_confidence_percent  visible_percent acquired_date acquired_time  green_1  green_2  green_3  green_4  green_5  green_6  green_7  green_8  green_9  green_10  green_11  green_12  green_13  green_14  green_15  green_16  green_17  green_18  overall_ndvi
20240604_162151_41_24d0 Feature 2024-06-04T16:21:51.419405Z                 0                        98            100          0.0              0            True  3.7                   0     PSB.SD   PSScene                   0                 3 planetscope 2024-06